In [ ]:
!pip install mtcnn
!pip install keras-facenet
!pip install opencv-python
!pip install scikit-learn

In [ ]:
import cv2
import os
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [ ]:
# file upload

from google.colab import files
files.upload()

In [ ]:
# Face detector Initialize

detector = MTCNN()
embedder = FaceNet()

In [ ]:
# Face extraction

def extract_face(image_path):
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  faces = detector.detect_faces(img)

  if len(faces) == 0:
    return None

  x, y, w, h = faces[0]['box']
  face = img[y:y+h, x:x+w]
  face = cv2.resize(face, (150, 150))

  return face

In [ ]:
!unzip -o /content/Faces.zip -d /content/

X = []
y = []

dataset_path = "/content/Faces"

for file_name in os.listdir(dataset_path):
  file_path = os.path.join(dataset_path, file_name)

  if os.path.isfile(file_path):
    face = extract_face(file_path)

    if face is not None:
      embedding = embedder.embeddings([face])[0]
      X.append(embedding)
      # Extract person's name
      person_name = file_name.split('_')[0]
      y.append(person_name)

X = np.array(X)
y = np.array(y)

In [ ]:
# Encoding

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [ ]:
# Model Train

model = SVC(kernel='linear', probability=True)
model.fit(X, y_encoded)

In [ ]:
# Model Save

import pickle

pickle.dump(model, open("face_recognition_model.pkl", "wb"))
pickle.dump(encoder, open("label_encoder.pkl", "wb"))

In [ ]:
# Face Recognition

def recognize_face(image_path):
  face = extract_face(image_path)

  if face is None:
    return "No Face Detected"

  embedding = embedder.embeddings([face])[0]
  embedding = embedding.reshape(1, -1)

  prediction = model.predict(embedding)
  confidence = model.predict_proba(embedding)

  name = encoder.inverse_transform(prediction)[0]
  conf = np.max(confidence)

  print(f"Recognized: {name}")
  print(f"Confidence: {conf:.2f}")

  plt.imshow(face)
  plt.show('off')

recognize_face("/content/Tom Cruise_45.jpg")